<a href="https://colab.research.google.com/github/amoyag/Bioquimica_Ing_Proteinas/blob/main/2-Estructura_pose/clase2-pose_score-alumnos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta.notebooks);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

# Working with Pose residues
Keywords: total_residue(), chain(), number(), pdb2pose(), pose2pdb(), conformation(), bond_length(), AtomID, atom_index(), pose_from_sequence(), bond_angle(), set_phi(), set_psi(), xyz(), score function, ScoreFunction(), get_score_function(), set_weight(), show(), etable_atom_pair_energies(), Atom objects, get_hbonds(), nhbonds(), residue_hbonds()

## Install and init PyRosetta

In [ ]:
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()

from pyrosetta import *
init()
#import os
# notebook_path = os.path.abspath("clase2-pose_score.ipynb")


## Load pdb files

In [ ]:
pdb_file = "PATH_TO_FILE/5tj3.pdb"
clean_pdb_file = "PATH_TO_FILE/5tj3.clean.pdb"
pose = pose_from_pdb(pdb_file)
pose_clean = pose_from_pdb(clean_pdb_file)

## Working with poses

   We can use methods in `Pose` to count residues and pick out residues from the pose. Remember that `Pose` is a python class, and to access methods it implements, you need an instance of the class (here `pose` or `pose_clean`) and you then use a dot after the instance.

In [ ]:
print(pose.total_residue())
print(pose_clean.total_residue())
# Did you catch all the missing residues before?

 Store the `Residue` information for residue 20 of the pose by using the `pose.residue(20)` function.

In [ ]:
# residue20 = type here
### BEGIN SOLUTION
residue20 =pose.residue(20)
### END SOLUTION
print(residue20.name())
print(residue20)


**Exercise: Residue objects**

Use the `pose`'s `.residue()` object to get the 24th residue of the protein pose. What is the 24th residue in the PDB file (look in the PDB file)? Are they the same residue?

In [ ]:
# store the 24th residue in the pose into a variable (see residue20 example above)
### BEGIN SOLUTION

### END SOLUTION


In [ ]:
# what other methods are attached to that Residue object? (type "residue24." and hit Tab to see a list of commands)

We can immediately see that the numbering PyRosetta internally uses for pose residues is different from the PDB file. The information corresponding to the PDB file can be accessed through the `pose.pdb_info()` object.

In [ ]:
print(pose.pdb_info().chain(24))
print(pose.pdb_info().number(24))
print(pose.residue(24).name())

In [ ]:
print(pose.residue(pose.pdb_info().pose2pdb(24)).name())

By using the `pdb2pose` method in `pdb_info()`, we can turn PDB numbering (which requires a chain ID and a residue number) into Pose numbering

In [ ]:
print(
    pose.residue(
    int(pose.pdb_info().pose2pdb(24).split()[0])
    ).name()
    )
# pose2pdb returns a string "residue_number chain_id", so we need to split the string and convert the residue number to an integer

In [ ]:
# PDB numbering to Pose numbering
print(pose.pdb_info().pdb2pose('A', 24))

Use the `pose2pdb` method in `pdb_info()` to see what is the corresponding PDB chain and residue ID for pose residue number 24

In [ ]:
# Pose numbering to PDB numbering

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

Now we can see how to examine the identity of a residue by PDB chain and residue number.

Once we get a residue, there are various methods in the `Residue` class that might be for running analysis. We can get instances of the `Residue` class from `Pose`. For instance, we can do the following:

In [ ]:
res_24 = pose.residue(24)
print(res_24.name())
print(res_24.is_charged())

## Getting spatial features from a Pose. Protein geometry

In [ ]:
from IPython.display import Image
Image('/content/google_drive/MyDrive/BIP_24-25/clase_2/dihedral-final.png',width='800')

`Pose` objects make it easy to access angles, distances, and torsions for analysis. Lets take a look at how to get backbone torsions first.

In [ ]:
#resid = "get the pose residue number for chain A:res 28 using the pdb2pose function"
### BEGIN SOLUTION

### END SOLUTION

Say we want to find the length of the $N$-$C_\alpha$ and $C_\alpha$-$C$ bonds for residue A:28 from the PDB file. We can use a couple approaches. The first involves using the bond length in the `Conformation` class, which stores some info on protein geometry. Take a look at some of the methods in the `Conformation` class using tab completion.

In [ ]:
conformation = pose.conformation()

Look at the documentation for the method `conformation.bond_length` below.

In [ ]:
### BEGIN SOLUTION
?conformation.bond_length

### END SOLUTION

To use the bond_length method in the `Conformation` class, it looks like we'll need to make `AtomID` objects. We can do this using an atom index and residue ID as follows:

In [ ]:
# Double Check: does resid contain the Pose numbering or PDB numbering?
res_28 = pose.residue(resid)
N28 = AtomID(res_28.atom_index("N"), resid)
CA28 = AtomID(res_28.atom_index("CA"), resid)
C28 = AtomID(res_28.atom_index("C"), resid)

# try printing out an AtomID object!

As usual, if you did not know how to construct an `AtomID`, you could check the documentation using `?AtomID`.

Now we can compute the bond lengths:

In [ ]:
print(pose.conformation().bond_length(N28, CA28))
print(pose.conformation().bond_length(CA28, C28))

Alternatively, we can compute bond lengths ourselves starting from the xyz coordinates of the atoms.

The method `xyz` of `Residue` returns a `Vector` class. The `Vector` class has various useful builtin methods including computing dot products, cross products, and norms. Through operator overloading in the `Vector` class, you can just subtract and add vector objects and they will manipulate the corresponding vectors appropriately.

In [ ]:
N_xyz = res_28.xyz("N")
CA_xyz = res_28.xyz("CA")
C_xyz = res_28.xyz("C")
N_CA_vector = CA_xyz - N_xyz
CA_C_vector = CA_xyz - C_xyz
print(N_CA_vector.norm())
print(CA_C_vector.norm())

Thankfully, the two approaches for computing distances check out!

**Note**: Not all bond lengths, angles, and torsions will be accessible using the `Conformation` object. That is because the `Conformation` object stores only the subset it needs to generate xyz locations for the atoms in the pose. The most stable way to get this information is to compute it using the xyz Cartesian coordinate vectors as a starting point.

We can check Rosetta's ideal $N$-$C_\alpha$ and $C_\alpha$-$C$ bond lengths. These ideal values would for instance be used if we generated a new pose from an amino acid sequence

In [ ]:
one_res_seq = "V" #Select valine to analise its geometry. This is not the valine in a particular protein.
pose_one_res = pose_from_sequence(one_res_seq)
print(pose_one_res.sequence())

In [ ]:
N_xyz = pose_one_res.residue(1).xyz("N")
CA_xyz = pose_one_res.residue(1).xyz("CA")
C_xyz = pose_one_res.residue(1).xyz("C")
print((CA_xyz - N_xyz).norm())
print((CA_xyz - C_xyz).norm())

Now lets figure out how to get angles in the protein. If the `Conformation` class has the angle we're looking for, we can use the AtomID objects we've already created:

In [ ]:
angle = pose.conformation().bond_angle(N28, CA28, C28)
print(angle) #This is the angle in radians

**Exercise**
Write some code to convert the angle above to degrees

In [ ]:
import math
angle*180/math.pi

In [ ]:
import math
# angle_degrees = math.degrees(angle)
print("Angle in degrees:", math.degrees(angle))

Based on the value of the N-$C_\alpha$-C angle. What is the geometry of the $C_\alpha$?

### Manipulating protein geometry

We can also alter the geometry of the protein, with particular interest in manipulating the protein backbone and $\chi$ dihedrals.

We will Perform each of the following manipulations on an alanine tripeptide, and see what happens with the coordinates of the CB atom of Pose residue 2 afterward.
- Set the $\phi$ of residue 2 to -60
- Set the $\psi$ of residue 2 to -43

In [ ]:
# three alanines
tripeptide = pose_from_sequence("AAA")

orig_phi = tripeptide.phi(2)
orig_psi = tripeptide.psi(2)
print("original phi:", orig_phi)
print("original psi:", orig_psi)

# print the xyz coordinates of the CB atom of residue 2 here BEFORE setting
### BEGIN SOLUTION

### END SOLUTION

In [ ]:
# set the phi and psi here
### BEGIN SOLUTION
tripeptide.set_phi(2, -60)
tripeptide.set_psi(2, -43)

print("new phi:", tripeptide.phi(2))
print("new psi:", tripeptide.psi(2))
### END SOLUTION

In [ ]:
# print the xyz coordinates of the CB atom of residue 2 here AFTER setting
### BEGIN SOLUTION
print("xyz coordinates:", tripeptide.residue(2).xyz("CB"))
### END SOLUTION
# did changing the phi and psi angle change the xyz coordinates of the CB atom of alanine 2?

### Pose visualization with PyMOL

We can inspect poses generated with PyRosetta using PyMOL. Start PyMOL and either use the PyMOL command line to run the PyMOL-RosettaServer.py file or drag and drop the PyMOL-RosettaServer.py file onto the PyMOL window to start the PyMOL-PyRosetta link.

The PyMOLMover class will let us send information from PyRosetta to PyMOL for quick visualization. We are creating an instance of PyMOLMover called pmm.

In [ ]:
from pyrosetta import PyMOLMover
pmm = PyMOLMover('127.0.0.1', 65000)
pmm.apply(pose_clean)

### Pose visualization with Py3Dmol

In [ ]:
!pip install Py3dmol

In [ ]:
import py3Dmol
#view = py3dmol.view(width=800, height=600)
v = py3Dmol.view()
v.addModel(open('PATH_TO_FILE/5tj3.clean.pdb').read())
v.setStyle({'cartoon': {'color': 'spectrum'}})  # Set the style to cartoon
v.zoomTo()  # Zoom to fit the structure
v.show()  #


## Rosetta Energy Score Functions


A basic function of Rosetta is calculating the energy or score of a biomolecule. This is important for inspecting the energies of a biomolecule at the whole protein, per-residue, and per-atom level.  

Rosetta has a standard energy function for all-atom calculations as well as several scoring functions for low-resolution protein representations. See https://www.ncbi.nlm.nih.gov/pubmed/28430426 for a review on the all-atom score functions.

You can also tailor an energy function by including scoring terms of your choice with custom weights.

To score a protein, you will begin by defining a score function using the `get_score_function(is_fullatom: bool)` method in the `pyrosetta.teaching` namespace. Specifying `True` will return the default `ref2015` all-atom energy function, while `False` will specify the default centroid score function.

Create a PyRosetta score function using:
```
sfxn = get_score_function(True)
```

In [ ]:
from pyrosetta.teaching import *

sfxn = get_score_function(True)

You can see the terms, weights, and energy method options by printing the score function:

```
print(sfxn)
```

In [ ]:
print(sfxn)

### Custom energy functions

You can also create a custom energy function that includes select terms. Typically, creating a whole new score function is unneccesary because the current one works well in most cases. However, tweaking the current energy function by reassigning weights and adding certain energy terms can be useful.

Here, we will make an example energy function with only the van der Waals attractive and repulsive terms, both with weights of 1. We need to use the `set_weight()`. Make a new `ScoreFunction` and set the weights accordingly. This is how we set the full-atom attractive (`fa_atr`) and the full-atom repulsive (`fa_rep`) terms.

```
sfxn2 = ScoreFunction()
sfxn2.set_weight(fa_atr, 1.0)
sfxn2.set_weight(fa_rep, 1.0)
```

In [ ]:
sfxn2 = ScoreFunction()
sfxn2.set_weight(fa_atr, 1.0)
sfxn2.set_weight(fa_rep, 1.0)

Lets compare the score of `pose_clean` using the full-atom `ScoreFunction` versus the `ScoreFunction` we made above using only the attractive and repulsive terms.

First, print the total energy of `pose_clean` using `print(sfxn(pose_clean))`
Then, print the attractive and repulsive energy only of `ras` using `print(sfxn2(pose_clean))`

In [ ]:
print(sfxn(pose_clean))

In [ ]:
print(sfxn2(pose_clean))

### Energy Breakdown
Using the full-atom `ScoreFunction` `sfxn`, break the energy of `pose_clean` down into its individual pieces with the `sfxn.show(pose_clean)` method. Which are the three most dominant contributions, and what are their values? Is this what you would have expected? Why? Note which terms are positive and negative

In [ ]:
sfxn.show(pose_clean)

In [ ]:
# Your response here: what are the three most dominant contributions?

Unweighted, individual component energies of each residue in a structure are stored in the `Pose` object and can be accessed by the `energies()` method. For example, to break down the energy into each residue's contribution, we use:
```
print(pose_clean.energies().show(<n>))
```
Where `<n>` is the residue number.

What are the total van der Waals, solvation, and hydrogen-bonding contributions of residue 24?

Note: The _backbone_ hydrogen-bonding terms for each residue are not available from the `Energies` object. You can get them by using EnergyMethodOptions. See http://www.pyrosetta.org/documentation#TOC-Hydrogen-Bonds-and-Hydrogen-Bond-Scoring.

In [ ]:
print(pose_clean.energies().show(24))

In [ ]:
# vdW = fa_atr + fa_rep =  -6.52 + 0.76; solvation = fa_sol = 6.43; hbond = hbond_sr_bb + hbond_lr_bb + hbond_bb_sc + hbond_sc = 0.00 + 0.00 + 0.00 + -0.66

The van der Waals, solvation, and electrostatic terms are atom-atom pairwise energies calculated from a pre-tabulated lookup table, dependent upon the distance between the two atoms and their types. You can access this lookup table, called the `etable` directly to check these energy calculations on an atom-by-atom basis. Use the `etable_atom_pair_energies` function which returns a triplet of energies for attractive, repulsive and solvation scores.

(Note that the `etable_atom_pair_energies()` function requires `Atom` objects, not the `AtomID` objects we saw earlier. For more info, look at the [documentation](https://graylab.jhu.edu/PyRosetta.documentation/pyrosetta.toolbox.atom_pair_energy.html?highlight=etable_atom_pair_energies#pyrosetta.toolbox.atom_pair_energy.etable_atom_pair_energies).)

**Practice:** What are the attractive, repulsive, solvation, and electrostatic components between the nitrogen of residue 24 and the oxygen of residue 20?


In [ ]:
res24 = pose_clean.residue(24)
res20 = pose_clean.residue(20)
res24_atomN = res24.atom_index("N")
res20_atomO = res20.atom_index("O")
pyrosetta.etable_atom_pair_energies(res24, res24_atomN, res20, res20_atomO, sfxn)

In [ ]:
help(pyrosetta.etable_atom_pair_energies)